# PileCore

______________________________________________________________________

**Authors: Thijs Lukkezen & Robin Wimmers**

This notebook shows how to perform a pile bearing calculation with PileCore for multiple CPTs.

The structure of the tutorial is as follows:

- [Project definition](#Input-definition)
- [Download CPT's in project](#Download-CPTs-in-project)
- [Classify CPTs](#Classify-CPTs)
- [Call PileCore-API](#Call-PileCore-API)
- [View Results](#View-Results)
- [Report](#Report)

As usual, we start to install and import the libraries that are needed for this notebook.

In [ ]:
# %matplotlib widget

import datetime
import os
import io
from typing import Any, Dict, Mapping, List
import logging

import numpy as np
import pandas as pd
import pygef
from matplotlib import pyplot as plt
from pygef.cpt import CPTData

from nuclei.client import NucleiClient
from tqdm import tqdm

from pypilecore import api, create_basic_pile
from pypilecore.common.piles.grid import PileGridProperties
from pypilecore.input.tension import (
    create_multi_cpt_payload,
    create_multi_cpt_report_payload,
)
from pypilecore.results import (
    MultiCPTTensionBearingResults,
    CasesMultiCPTBearingResults,
)

from pypilecore.viewers import (
    ViewerCptResults,
    ViewerCptResultsPlanView,
)

pd.set_option("display.max_columns", None)
logging.getLogger().setLevel(logging.INFO)

### Start a Nuclei client session

In the next cell we will create a nuclei-client with a session that takes care of the
authentication and communication with the Nuclei server.

You will need to provide your user token, which can be obtained by login in to the [nuclei website](https://nuclei.cemsbv.io/) with your personal credentials and going to the "API Access Tokens" section.

<div style="background: #f2ed4c;
            width: 100%;
            color: black;
            text-align: center;">
<b>USER INPUT REQUIRED BELOW:<b>
</div>

In [ ]:
# os.environ["NUCLEI_TOKEN"] = "<YOUR TOKEN>"

client = NucleiClient()

# Input definition

#### General input

In [ ]:
# General input data
project_id = 21305
project_remark = "Voorbeeld Notebook"  # Optional additional information
author = "N. Uclei"
project_name = "Automated pile design"

#### Soil Investigation input

In [ ]:
# Specify CPT selection

# ** cpt_selection
# Specify a list of CPTs names (i.e. BRO ID).
cpt_selection = [
    "CPT000000059020",
    "CPT000000061575",
    "CPT000000061576",
    "CPT000000061577",
    "CPT000000061578",
    "CPT000000061579",
    "CPT000000061580",
    "CPT000000064642",
]


# ** classify_metode:
# Metode used to classify CPT data.
# Accepted values: ["beenJefferies", "machineLearning", "nen", "table", "robertson", "ntype"]
classify_metode = "ntype"

#### Geometry input

In [ ]:
# ** pile_tip_levels_nap
# These are the levels (w.r.t. NAP) at which the bearing capacities will be calculated
# It can be a list, or any other sequence, such as a numpy array.
pile_tip_levels_nap = np.arange(-15, -25, -0.5)

# ** pile_head_level_nap
# The level of the pile-head [m] w.r.t. NAP.
# Must be a number, or the string "surface". In the latter case, the pile-head will be
# situated at the level of the original CPT surface level.
pile_head_level_nap = "surface"

# ** groundwater_level
# The groundwater-level in the project (w.r.t. NAP). Will have an impact on the
# classification and on the calculated soil-stresses.
# When None, the water-level of the CPTs is used.
groundwater_level_nap = -5

#### Excavation input

In [ ]:
# ** excavation_depth_nap:
# The depth [m w.r.t. NAP] of the service-level after excavation.
# Has to be below the origin al service-level of the CPT.
excavation_depth_nap = None

# ** excavation_param_t:
# Required when providing an excavation_depth. The values can be:
# - 1.0: if installation is not low in vibration (niet-trillingsarm) and piles are installed after excavating
# - 0.5: (wortel-methode) if piles have been installed before excavation or installation is low-vibrating
excavation_param_t = 0.5

# ** excavation_stress_reduction_method:
# Method used to calculate the stress reduction due to the excavation applied to the effective and total stresses.
# Only used when `excavation_depth_nap` is different than `None`. It can be:
# - "constant": The stress reduction below the excavation is constant with depth. The stress reduction is equal to
#       the original effective stress (i.e. before the excavation) at the excavation depth.
# - "begemann": The stress reduction below the excavation decreases with depth according to the Begemann method.
excavation_stress_reduction_method = "constant"

# ** excavation_width:
# Width of the excavation [m].
# Used to calculate the stress reduction due to the excavation if the Begemann method is selected.
# Accepted values: > 0.0
excavation_width = None

# ** excavation_edge_distance:
# Distance from the edge of the excavation to the pile centerline [m].
# Used to calculate the stress reduction due to the excavation if the Begemann method is selected.
# Accepted values: >= 0.0 and <= 0.5 * excavation_width
# Note: 0.0 means pile at edge of excavation, 0.5 * excavation_width means pile at center of excavation.
excavation_edge_distance = None

#### Construction and loads input

In [ ]:
# ** construction_sequence:
# Value that indicates if CPT are performed before or after pile
# installation according to 7.3.1 CUR 2001-4.
# Accepted values: ["cpt-pile", "pile-cpt"]
# Notes:
# - If standard "NEN9997-1" is used the following applies:
#       - `cpt-pile`, then the `f1` factor is computed according to NEN 9997-1+C2:2017 7.6.3.3(e).
#       - `pile-cpt`, then the `f1` factor is 1.0 at all depths.
# - If standard "CUR236" is used `f1` is always 1.0 regardless of the
#   construction sequence.
construction_sequence = "cpt-pile"

# ** stiff_construction:
# The stiffness of the construction has an influence on the value of xi.
# Accepted values: [True, False]
stiff_construction = False

# ** pile_load_sls_max
# Maximum tension force on pile Ft;max;k. Note that only positive
# values (tension force) are accepted.
pile_load_sls_max = 350

# ** pile_load_sls_min
# Minimum tension force (or maximum compression force) on pile
# -Ft;min;k (tension > 0). Positive values (tension force), negative
# values (compression force) and 0.0 are accepted. Note that the
# positive must be <= `pile_load_sls_max`
pile_load_sls_min = 50

# ** soil_load_sls:
# The overburden-pressure at surface-level [kPa]
# If None, the default is 0.0
soil_load_sls = 0

#### top of tension zone

In [ ]:
# ** top_of_tension_zone_nap
# Level (w.r.t. NAP) for with the bearing capacity we be calcualted.
# If None top_of_tension_zone_nap will be pile_head_level_nap.
# Please note that chaging the top_of_tension_zone_nap have in impact on the
# pile settelement calculation. All values above this level will be ignored resulting
# in a lower pile displacement.
# !! PileCore will correct for remarks regarding L;a for CUR236 6.1.1. !!
top_of_tension_zone_nap = -5.0

# ** Overrule top_of_tension_zone_nap for specific CPTs
# If desired, you can also specify top_of_tension_zone_nap values per CPT
# The values provided below will overwrite the top_of_tension_zone_nap for those specific CPTs.

# ** individual_top_of_tension_zone_nap
# dictionary with key cpt name and top_of_tension_zone_nap value
# e.g.: {"S03": 1.5}
individual_top_of_tension_zone_nap: Mapping[Any, float] = {}

#### OCR input

In [ ]:
# ** ocr
# The Over-Consolidation-Ratio [-] of the foundation layer.
# If None, an OCR of 1.0 is assumed.
ocr = None

# ** Overrule OCR for specific CPTs
# If desired, you can also specify OCR values per CPT
# The values provided below will overwrite the OCR for those specific CPTs.

# ** individual_ocr
# dictionary with key cpt name and OCR value
# e.g.: {"S03": 1.5}
individual_ocr: Mapping[Any, float] = {}

#### Safety Factors input

In [ ]:
# ** gamma_gamma
# Partial factor for volumetric soil weight. A.3.2 NEN 9997-1+C2 (nl)
# Geotechnisch ontwerp van constructies - Deel 1 Algemene regels [2017]
# If None, the default is 1.1
gamma_gamma = 1.1

# ** gamma_r_s
# Safetyfactor on the sleeve-friction bearing capacity
# If None, the default is 1.2
gamma_r_s = 1.2

# ** gamma_r_b
# Safetyfactor on the pile-tip bearing capacity
# If None, the default is 1.2
gamma_r_b = 1.2

# ** gamma_f_nk
# Safetyfactor on the negative friction
# Note: Use 1.4 if bottom negative friction ≠ positive friction
# If None, the default is 1.0
gamma_f_nk = 1.0

# ** gamma_s_t
# Pile resistance factor gamma_s;t used to compute the
# design cone resistance values qc;z;d as prescribed in NEN 9997-1+C2_2017 7.6.3.3(d).
# If None, the default is 1.35
gamma_s_t = 1.35

# ** overrule_xi
# Sets a fixed value for xi
overrule_xi = None

## Pile input
---


In [ ]:
# ** pile_name
# The name of the pile (For potential plotting purposes)
pile_name = None

### Standard piles

Choose a `main_type`, `specification` and `installation` (optional) value to define a standard pile according to the NEN9997-1 or CUR236 definitions.

You can also leave the standard pile definition empty and define all pile attributes manually.

#### NEN9997-1
![NEN9997-1a](./img/standard_piles-NEN9997-1.png)

#### CUR-236
![CUR236a](./img/standard_piles-CUR236.png)

In [ ]:
# Standard Pile definition

# ** standard
# The choice of "standard" from one of the tables below.
# Accepted values: ["NEN9997-1", "CUR236"]
standard = "CUR236"

# ** main_type
# The choice of "main_type" from one of the tables below.
# Accepted values: ["concrete", "steel", "wood", "anchor"]
# when standard == CUR236 only anchor type piles are accepted
main_type = "anchor"

# ** specification
# The "specification" section in the table below.
specification = "3"

# ** installation
# The English-equivalent of the "Installatie" section in the table below.
installation = None

### Pile Geometry
![pile_geometry_definitions](./img/pile_geometry_definitions.png)

In [ ]:
# ** pile_shape
# The shape of the pile
# Accepted values: ["round", "rectangle"]
pile_shape = "round"

# ** height_base
# Height of pile base [m]. If None, a pile with constant dimension is inferred.
# When standard == CUR236 height_base should be None, making the shaft dimensions equal to the base dimensions.
height_base = None

### Rectangular pile dimensions

Only fill these values if the `pile_shape` == "rectangle". Otherwise they are ignored

The pile geometry is defined as a "core" and "shell" segment. First we define the "core",
which is a required part that should always be there for a rectangular pile.

In [ ]:
# ** core_secondary_dimension
# Largest cross-sectional dimension of the core component [m].
core_secondary_dimension = 0.40

# ** core_tertiary_dimension
# Smallest cross-sectional dimension of the core component [m].
# If None, a square core is inferred.
core_tertiary_dimension = None

# Base dimensions (optional)
# --------------------------
# Optionally, we can define a "base" segment. This segment should have larger dimensions than the core.
# Make sure to also fill a value for the `height_base` (in the general geometry cell above).

# ** base_secondary_dimension
# Largest cross-sectional dimension of the widened-base component [m].
base_secondary_dimension = None

# ** base_tertiary_dimension
# Smallest cross-sectional dimension of the widened-base component [m].
# If None, a square base is inferred.
base_tertiary_dimension = None

### Round pile dimensions

Only fill these values if the `pile_shape` == "round". Otherwise they are ignored

The pile geometry is defined as a "core" and "shell" segment. First we define the "core",
which is a required part that should always be there for a round pile.

In [ ]:
# ** core_diameter
# Diameter of the core component [m].
core_diameter = 0.22

# Base dimension (optional)
# --------------------------
# Optionally, we can define a "base" segment. This segment should have larger dimensions than the core.
# Make sure to also fill a value for the `height_base` (in the general geometry cell above).
# Please note that when standard == CUR236 that base dimension is used for the grout shell and must
# be provided.

# ** base_diameter
# Diameter of the widened-base [m].
# If null, a pile with constant diameter (diameter_base) is inferred.
base_diameter = 0.40

### Optionally overwriteable pile input

If you want to use the default pile specifications you can ignore this section. Any
value provided will overwrite the default value.

In [ ]:
# Optional Input

# ** pile_material
# The material name of the pile. If a standard pile was selected, the material is
# inferred, but can be overwritten. Default materials are: "concrete", "steel", "wood".
# Custom materials can be defined below.
pile_material = None

# ** custom_material
# A custom material definition. Assign the "name" property as `pile_material` to use it.
# example:
# custom_material = {
#     "name": "custom_material",
#     "elastic_modulus": 15e3,    # [MPa]
#     "color": "#ff0000",        # Hexadecimal color
# }
custom_material = None

# ** settlement_curve
# Settlement lines for figures 7.n and 7.o of NEN-9997-1 As defined in table 7.c of
# NEN-9997-1. The value is inferred from the pile_type_specifications, but can be
# overwritten
settlement_curve = None

# ** adhesion
# Optional adhesion value, use it if the pile shaft has undergone a special treatment.
# Examples:
# - adhesion = 50 kN/m2 for synthetic coating
# - adhesion = 20 kN/m2 for bentonite
# - adhesion = 10 kN/m2 for bitumen coating
# See 7.3.2.2(d) of NEN 9997-1 for examples.
adhesion = None  # kPa

# ** alpha_p
# Alpha p factor used in pile tip resistance calculation. The value is inferred from the
# pile_type_specifications, but can be overwritten.
alpha_p = None

# ** alpha_s_clay
# Alpha s factor for soft layers used in the positive friction calculation. If None the
# factor is determined as specified in table 7.d of NEN 9997-1.
alpha_s_clay = None

# ** alpha_s_sand
# Alpha s factor for coarse layers used in the positive friction calculation. The value
# is inferred from the pile_type_specifications, but can be overwritten.
alpha_s_sand = None

# ** alpha_t_clay
# Alpha t factor for soft layers used in the positive friction calculation. If None the
# factor is determined as specified in table 7.d of NEN 9997-1.
alpha_t_clay = None

# ** alpha_t_sand
# Alpha t factor for coarse layers used in the positive friction calculation. The value
# is inferred from the pile_type_specifications, but can be overwritten.
alpha_t_sand = None

# ** beta_p
# Beta_p used in pile tip resistance calculation as per NEN 9997-1 7.6.2.3 (h). The
# value is inferred from the pile dimension properties, but can be overwritten
beta_p = None

# ** pile_tip_factor_s
# Factor s used in pile tip resistance calculation as per NEN 9997-1 7.6.2.3 (h). The
# value is inferred from the pile dimensions and soil properties, but can be overwritten.
pile_tip_factor_s = None

# ** is_auger
# Determines weather the pile the pile is an auger pile or not. The value is inferred
# from the pile_type_specifications, but can be overwritten.
# Accepted values: [True, False, None]
is_auger = None

# ** is_low_vibrating
# Determines weather the pile has an installation type with low vibration. The value is
# inferred from the pile_type_specifications, but can be overwritten.
# Accepted values: [True, False, None]
is_low_vibrating = None

# ** negative_fr_delta_factor
# factor * φ = δ. This parameter will be multiplied with phi to get the delta parameter
# used in negative friction calculation according to NEN-9997-1 7.3.2.2 (e). Typically
# values are 1.0 for piles cast in place, and 0.75 for other pile types. The value is
# inferred from the pile_type_specifications, but can be overwritten.
negative_fr_delta_factor = None

# ** qc_z_a_lesser_1m
# Maximum cone resistance `qc` value allowed for layers with thickness < 1m in
# the calculation of positive skin friction resistance. This value is used to compute the
# trimmed (chamfered) `qc;z;a` values according to NEN 9997-1+C2:2017 7.6.2.3.(10)(i).
# It must be less or equal than `qc_z_a_greater_1m`. If None, then 12 MPa is used.
# This attribute only applies to the NEN-9997 pile types and standards.
qc_z_a_lesser_1m = None

# ** qc_z_a_greater_1m
# Maximum cone resistance `qc` value allowed for layers with thickness >= 1m in
# the calculation of positive skin friction resistance. This value is used to compute the
# trimmed (chamfered) `qc;z;a` values according to NEN 9997-1+C2:2017 7.6.2.3.(10)(i)
# It must be greater or equal than `qc_z_a_lesser_1m`. If None, then 15 MPa is used.
# This attribute only applies to the NEN-9997 pile types and standards.
qc_z_a_greater_1m = None

# ** chamfered
# The chamfered value can be overwritten by the user [MPa].
# This attribute only applies to the CUR-236 pile types and standards.
chamfered = None

### Create Pile

In [ ]:
pile = create_basic_pile(
    pile_name=pile_name,
    main_type=main_type,
    specification=specification,
    installation=installation,
    pile_shape=pile_shape,
    height_base=height_base,
    core_secondary_dimension=core_secondary_dimension,
    core_tertiary_dimension=core_tertiary_dimension,
    base_secondary_dimension=base_secondary_dimension,
    base_tertiary_dimension=base_tertiary_dimension,
    core_diameter=core_diameter,
    base_diameter=base_diameter,
    pile_material=pile_material,
    custom_material=custom_material,
    settlement_curve=settlement_curve,
    adhesion=adhesion,
    alpha_p=alpha_p,
    alpha_s_clay=alpha_s_clay,
    alpha_s_sand=alpha_s_sand,
    alpha_t_clay=alpha_t_clay,
    alpha_t_sand=alpha_t_sand,
    beta_p=beta_p,
    pile_tip_factor_s=pile_tip_factor_s,
    is_auger=is_auger,
    is_low_vibrating=is_low_vibrating,
    negative_fr_delta_factor=negative_fr_delta_factor,
    qc_z_a_lesser_1m=qc_z_a_lesser_1m,
    qc_z_a_greater_1m=qc_z_a_greater_1m,
    chamfered=chamfered,
)

In [ ]:
pile.geometry.plot();

### Other input

Some other input values.

In [ ]:
# ** void_ratio_max
# Maximum void ratio of the soil (the loosest packing). The influence
# of this parameter is limited, and therefore it is typically
# sufficient to provide a global estimation. For normally consolidated
# sands in The Netherlands, an emax = 0.80 can be used in most cases.
void_ratio_max = 0.8

# ** void_ratio_min
# Minimum void ratio of the soil (the densest packing). The influence
# of this parameter is limited, and therefore it is typically
# sufficient to provide a global estimation. For normally consolidated
# sands in The Netherlands, an emin = 0.40 can be used in most cases.
void_ratio_min = 0.4

In [ ]:
# ** center_to_center_distance
# Centre to centre distance of regular grid [m]
# if None pile is calculated as a single pile
center_to_center_distance = None

# ** pile_location
# - center pile `pile_location= 4`
# - middle pile `pile_location= 1 or 3 or 4 or 5 or 7`
# - corner pile `pile_location= 0 or 2 or 6 or 8`

#     6 --- 7 --- 8
#     |     |     |
#     3 --- 4 --- 5
#     |     |     |
#     0 --- 1 --- 2   with --- is | is center_to_center_distance
pile_location = 4

In [ ]:
if center_to_center_distance:
    pile_grid = PileGridProperties.regular(
        ctc=center_to_center_distance, index_location=pile_location
    )
    pile_grid.plot_overview()
else:
    pile_grid = None

### Report content

These values define the content of the report

In [ ]:
# ** group_results_content
# Whether or not to add a section with the results of all CPTs considered as one
# statistical group.
# Accepted values: [True, False]
group_results_content = True

# ** individual_cpt_results_content
# Whether or not to add a separate result section for each individual CPT.
# Accepted values: [True, False]
individual_cpt_results_content = True

# ** result_summary_content
# Whether or not to add a summary of all results in the beginning of the report.
# Accepted values: [True, False]
result_summary_content = True

<div style="background: #f2ed4c;
            width: 100%;
            color: black;
            text-align: center;">
<b>END USER INPUT<b>
</div>

After this point, modifications are for expert users

#### Download CPTs in project

In [ ]:
# Get CPTs
# loop over the cpt id's and fetch file from BRO
cptdata_objects: List[CPTData] = []
for file_metadata in tqdm(cpt_selection, desc="Download CPT's from BRO"):
    # download CPT from BRO
    response = client.session.get(
        url=f"https://publiek.broservices.nl/sr/cpt/v1/objects/{file_metadata}"
    )
    if not response.ok:
        print(
            f"RuntimeError: {file_metadata} could not be donwloaded from de BRO server. \n Statuse code: {response.status_code}"
        )
        continue

    cpt = pygef.read_cpt(io.BytesIO(response.content))
    object.__setattr__(cpt, "alias", file_metadata)
    cptdata_objects.append(cpt)

#### Classify CPTs

In [ ]:
classify_tables: Dict[str, dict] = {}

for i, cpt in tqdm(enumerate(cptdata_objects), desc="Classify CPT's"):
    # remove nan data
    data = cpt.data.drop_nulls()

    # classify CPT with CPTCore
    payload = {
        "aggregateLayersPenalty": 5,
        "minimumSegmentLength": 5,
        "data": {
            "coneResistance": data.get_column("coneResistance").clip(0, 50).to_list(),
            "correctedPenetrationLength": data.get_column("depth").to_list(),
            "localFriction": data.get_column("localFriction").clip(0, 50).to_list(),
        },
        "verticalPositionOffset": cpt.delivered_vertical_position_offset,
        "x": cpt.delivered_location.x,
        "y": cpt.delivered_location.y,
    }
    if "porePressureU2" in data.columns:
        payload["data"]["porePressureU2"] = (
            data.get_column("porePressureU2").clip(0, 50).to_list(),
        )[0]

    response = client.session.post(
        f"https://crux-nuclei.com/api/cptcore/v1/classify/{classify_metode}",
        json=payload,
    )
    if not response.ok:
        cptdata_objects.pop(i)
        print(
            f"RuntimeError: {file_metadata} could not be classified. \n Statuse code: {response.status_code}"
        )
        continue
    classify_tables[cpt.alias] = response.json()

#### Call PileCore-API

In [ ]:
# Get results

multi_cpt_payload, results_passover = create_multi_cpt_payload(
    cptdata_objects=cptdata_objects,
    classify_tables=classify_tables,
    groundwater_level_nap=groundwater_level_nap,
    excavation_depth_nap=excavation_depth_nap,
    pile=pile,
    excavation_param_t=excavation_param_t,
    excavation_stress_reduction_method=excavation_stress_reduction_method,
    excavation_width=excavation_width,
    excavation_edge_distance=excavation_edge_distance,
    pile_head_level_nap=pile_head_level_nap,
    pile_tip_levels_nap=pile_tip_levels_nap,
    gamma_f_nk=gamma_f_nk,
    gamma_r_b=gamma_r_b,
    gamma_r_s=gamma_r_s,
    gamma_s_t=gamma_s_t,
    gamma_gamma=gamma_gamma,
    overrule_xi=overrule_xi,
    void_ratio_max=void_ratio_max,
    void_ratio_min=void_ratio_min,
    pile_load_sls_max=pile_load_sls_max,
    pile_load_sls_min=pile_load_sls_min,
    soil_load_sls=soil_load_sls,
    stiff_construction=stiff_construction,
    ocr=ocr,
    individual_ocr=individual_ocr,
    pile_grid=pile_grid,
    top_of_tension_zone_nap=top_of_tension_zone_nap,
    individual_top_of_tension_zone_nap=individual_top_of_tension_zone_nap,
    construction_sequence=construction_sequence,
)

api_response = api.get_multi_cpt_api_result_tension(
    client=client, payload=multi_cpt_payload, standard=standard
)

multi_bearing_results = MultiCPTTensionBearingResults.from_api_response(
    response_dict=api_response,
    cpt_input=results_passover,
)

## View Results

In [ ]:
# plot the bearing capacities for the CPTs as a single group
multi_bearing_results.group_results_table.plot_bearing_capacities();

In [ ]:
# Get the results table for the CPT group

multi_bearing_results.group_results_table.to_pandas().round(2)

Results for individual CPTs

In [ ]:
R_t_d_kluit = multi_bearing_results.cpt_results.get_results_per_cpt(
    column_name="R_t_d_plug"
).round(1)
# R_t_d_kluit.to_csv(f"{project_name} Kluidgewicht.csv")

print("Kluidgewicht")
R_t_d_kluit

In [ ]:
R_t_d = multi_bearing_results.cpt_results.get_results_per_cpt(
    column_name="R_t_d"
).round(1)
# R_t_d.to_csv(f"{project_name} Schachtweerstand.csv")

print("Schachtweerstand")
R_t_d

## Interactive Viewers

In [ ]:
# Optionally define a case name for the results (to be shown in the interactive viewers)
case_name = None

In [ ]:
# Parse the results into a CasesMultiCPTBearingResults
results_per_case = {
    case_name: multi_bearing_results,
}

cpt_locations = {cpt.alias: cpt.delivered_location for cpt in cptdata_objects}

cases_multi_results = CasesMultiCPTBearingResults(
    results_per_case=results_per_case, cpt_locations=cpt_locations
)

Viewer Cpt Results

In [ ]:
viewer_cpt_results = ViewerCptResults(results_cases=cases_multi_results)
viewer_cpt_results.display()

Viewer Cpt Results Plan View

In [ ]:
viewer_cpt_results_plan_view = ViewerCptResultsPlanView(
    results_cases=cases_multi_results
)
viewer_cpt_results_plan_view.display()

#### Single CPT inspection

It's possilbe to select one CPT from the group result. This object holds all the data releated to a singel CPT, like soil table and coneResistance.

In [ ]:
# Get the available CPT names
multi_bearing_results.cpt_names

In [ ]:
# Select a CPT test-id to inspect
single_cpt_result = multi_bearing_results.cpt_results["CPT000000059020"]

In [ ]:
# Get pandas dataframe of single-cpt results

single_cpt_result.table.to_pandas()

In [ ]:
single_cpt_result.pile_grid_properties.plot_overview();

## Report

In [ ]:
# Create report

# Close all open plots to save memory
plt.close("All")

multi_cpt_report_payload = create_multi_cpt_report_payload(
    multi_cpt_payload=multi_cpt_payload,
    project_name=project_name,
    project_id=str(project_id),
    author=author,
    date=datetime.date.today().strftime("%d-%m-%y"),
    group_results_content=group_results_content,
    individual_cpt_results_content=individual_cpt_results_content,
    result_summary_content=result_summary_content,
)

report = api.get_multi_cpt_api_report_tension(
    client=client, payload=multi_cpt_report_payload, standard=standard
)

with open(f"{project_name}_report.pdf", "wb") as f:
    f.write(report)